In [1]:
import json
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf

print(sys.version)
print(np.__version__)
print(sklearn.__version__)
print(tf.__version__)

3.7.10 (default, Feb 26 2021, 18:47:35) 
[GCC 7.3.0]
1.18.5
0.24.1
1.15.5


## Deep Bayesian Bandits Reproducibility

This notebook explores the reproducibility around the [Deep Bayesian Bandits](https://github.com/tensorflow/models/tree/archive/research/deep_contextual_bandits) work by Google. We look at the LinTS implementation, which forms the baseline of their experiments.

In order to run these experiments, please perform the steps below:
- Clone the [tensorflow models repo](https://github.com/tensorflow/models), switch to the `archive` branch, and copy `models/research/deep_contextual_bandits/bandits` folder to this directory.
- Run the cell below to overwrite the LinTS implementation file. This updates the multivariate sampling such that we can select the method to use while sampling, between SVD and Cholesky. Note that the SVD method was used in the original code.

In [2]:
%%writefile bandits/algorithms/linear_full_posterior_sampling.py
# Copyright 2018 The TensorFlow Authors All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Contextual algorithm that keeps a full linear posterior for each arm."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from scipy.stats import invgamma

from bandits.core.bandit_algorithm import BanditAlgorithm
from bandits.core.contextual_dataset import ContextualDataset


class LinearFullPosteriorSampling(BanditAlgorithm):
  """Thompson Sampling with independent linear models and unknown noise var."""

  def __init__(self, name, hparams):
    """Initialize posterior distributions and hyperparameters.

    Assume a linear model for each action i: reward = context^T beta_i + noise
    Each beta_i has a Gaussian prior (lambda parameter), each sigma2_i (noise
    level) has an inverse Gamma prior (a0, b0 parameters). Mean, covariance,
    and precision matrices are initialized, and the ContextualDataset created.

    Args:
      name: Name of the algorithm.
      hparams: Hyper-parameters of the algorithm.
    """

    self.name = name
    self.hparams = hparams
    self.rng = np.random.default_rng(self.hparams.seed)

    # Gaussian prior for each beta_i
    self._lambda_prior = self.hparams.lambda_prior

    self.mu = [
        np.zeros(self.hparams.context_dim + 1)
        for _ in range(self.hparams.num_actions)
    ]

    self.cov = [(1.0 / self.lambda_prior) * np.eye(self.hparams.context_dim + 1)
                for _ in range(self.hparams.num_actions)]

    self.precision = [
        self.lambda_prior * np.eye(self.hparams.context_dim + 1)
        for _ in range(self.hparams.num_actions)
    ]

    # Inverse Gamma prior for each sigma2_i
    self._a0 = self.hparams.a0
    self._b0 = self.hparams.b0

    self.a = [self._a0 for _ in range(self.hparams.num_actions)]
    self.b = [self._b0 for _ in range(self.hparams.num_actions)]

    self.t = 0
    self.data_h = ContextualDataset(hparams.context_dim,
                                    hparams.num_actions,
                                    intercept=True)

  def action(self, context):
    """Samples beta's from posterior, and chooses best action accordingly.

    Args:
      context: Context for which the action need to be chosen.

    Returns:
      action: Selected action for the context.
    """

    # Round robin until each action has been selected "initial_pulls" times
    if self.t < self.hparams.num_actions * self.hparams.initial_pulls:
      return self.t % self.hparams.num_actions

    # Sample sigma2, and beta conditional on sigma2
    sigma2_s = [
        self.b[i] * invgamma.rvs(self.a[i])
        for i in range(self.hparams.num_actions)
    ]

    try:
        if self.hparams.method == 'default':
            beta_s = [
              np.random.multivariate_normal(self.mu[i], sigma2_s[i] * self.cov[i])
              for i in range(self.hparams.num_actions)
            ]
        else:
            beta_s = [
                self.rng.multivariate_normal(self.mu[i], sigma2_s[i] * self.cov[i], method=self.hparams.method)
                for i in range(self.hparams.num_actions)
            ]
    except np.linalg.LinAlgError as e:
      # Sampling could fail if covariance is not positive definite
      print('Exception when sampling from {}.'.format(self.name))
      print('Details: {} | {}.'.format(e.message, e.args))
      d = self.hparams.context_dim + 1
      beta_s = [
          np.random.multivariate_normal(np.zeros((d)), np.eye(d))
          for i in range(self.hparams.num_actions)
      ]

    # Compute sampled expected values, intercept is last component of beta
    vals = [
        np.dot(beta_s[i][:-1], context.T) + beta_s[i][-1]
        for i in range(self.hparams.num_actions)
    ]

    return np.argmax(vals)

  def update(self, context, action, reward):
    """Updates action posterior using the linear Bayesian regression formula.

    Args:
      context: Last observed context.
      action: Last observed action.
      reward: Last observed reward.
    """

    self.t += 1
    self.data_h.add(context, action, reward)

    # Update posterior of action with formulas: \beta | x,y ~ N(mu_q, cov_q)
    x, y = self.data_h.get_data(action)

    # The algorithm could be improved with sequential update formulas (cheaper)
    s = np.dot(x.T, x)

    # Some terms are removed as we assume prior mu_0 = 0.
    precision_a = s + self.lambda_prior * np.eye(self.hparams.context_dim + 1)
    cov_a = np.linalg.inv(precision_a)
    mu_a = np.dot(cov_a, np.dot(x.T, y))

    # Inverse Gamma posterior update
    a_post = self.a0 + x.shape[0] / 2.0
    b_upd = 0.5 * (np.dot(y.T, y) - np.dot(mu_a.T, np.dot(precision_a, mu_a)))
    b_post = self.b0 + b_upd

    # Store new posterior distributions
    self.mu[action] = mu_a
    self.cov[action] = cov_a
    self.precision[action] = precision_a
    self.a[action] = a_post
    self.b[action] = b_post

  @property
  def a0(self):
    return self._a0

  @property
  def b0(self):
    return self._b0

  @property
  def lambda_prior(self):
    return self._lambda_prior


Overwriting bandits/algorithms/linear_full_posterior_sampling.py


We replicate the [quick start example](https://github.com/tensorflow/models/blob/archive/research/deep_contextual_bandits/example_main.py) from the Deep Contextual Bandits research repo below, focusing on just LinTS, and evaluating the cumulative reward at the end. Note that all the seeds are set, such that the cumulative reward should be the same when the code is run in the same environment.

In [3]:
import os
import sys
import time

import numpy as np
import tensorflow as tf

from tensorflow.python.util import deprecation

from bandits.data.data_sampler import sample_mushroom_data
from bandits.core.contextual_bandit import run_contextual_bandit
from bandits.algorithms.linear_full_posterior_sampling import LinearFullPosteriorSampling

if type(tf.contrib) != type(tf):
    tf.contrib._warning = None

def sample_data(num_contexts):
    num_actions = 2
    context_dim = 117
    file_name = 'mushroom.data'
    dataset, opt_mushroom = sample_mushroom_data(file_name, num_contexts)
    opt_rewards, opt_actions = opt_mushroom
    return dataset, opt_rewards, opt_actions, num_actions, context_dim

# Problem parameters
num_contexts = 2000

# Create dataset
np.random.seed(42)
sampled_vals = sample_data(num_contexts)
dataset, opt_rewards, opt_actions, num_actions, context_dim = sampled_vals

def run_dbb(random_option):
    np.random.seed(42)
    hparams_linear = tf.contrib.training.HParams(num_actions=num_actions,
                                                 context_dim=context_dim,
                                                 a0=6,
                                                 b0=6,
                                                 lambda_prior=0.25,
                                                 initial_pulls=2,
                                                 seed=42,
                                                 method=random_option)

    algos = [
          LinearFullPosteriorSampling('LinFullPost', hparams_linear),
    ]
    t_init = time.time()
    results = run_contextual_bandit(context_dim, num_actions, dataset, algos)
    _, h_rewards = results


    reward = np.sum(h_rewards[:, 0])
    return reward

### Option 1
We use the default implementation, which uses `np.random.multivariate_random`, and set the global seed to ensure reproducibility in a single environment. Note that this is the same as using `np.random.RandomState`, as the global seed sets the random state.

In [4]:
default_reward = run_dbb('default')
default_reward

3475.0

### Option 2
We use the new `Generator` class with default parameters, which internally uses SVD for decomposition:

In [5]:
svd_reward = run_dbb('svd')
svd_reward

3540.0

### Option 3
We use Cholesky decomposition with the new Generator class. Our hypothesis is that this will produce reproducible results across different environments.

In [6]:
cholesky_reward = run_dbb('cholesky')
cholesky_reward

4140.0

We save all the results for analysis.

In [7]:
rewards = pd.DataFrame({
    'env': ['LinuxUbuntu_MKL'],
    'default': [default_reward],
    'svd': [svd_reward],
    'cholesky': [cholesky_reward],
}).set_index('env')
rewards

,default,svd,cholesky
env,,,
LinuxUbuntu_MKL,3475.0,3540.0,4140.0


In [8]:
os.makedirs('output', exist_ok=True)
rewards.to_csv(os.path.join('output', 'linuxubuntu_mkl_rewards.csv'))